In [17]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

In [25]:
from py2neo import Graph
from transformers import AutoTokenizer, AutoModel
import torch

# Kết nối tới cơ sở dữ liệu Neo4j
neo4j_graph = Graph("neo4j+s://503c5eaf.databases.neo4j.io", auth=("neo4j", "OplNn6X6ehDa3GhvqJECbHcfXSe_L9juCTK2SGuvDaQ"), name="neo4j")

# Truy vấn lấy thông tin các môn học
query = """
  MATCH (ancestor:Resource {uri: 'http://localhost/ontologies/2024/10/11/edu_program#monHoc'})
  MATCH (n:Resource)-[:rdfs__subClassOf*]->(ancestor)
  MATCH (instance:Resource)-[:rdf__type]->(n)
  RETURN DISTINCT instance.ns0__maMonHoc AS code, instance.rdfs__label AS label, id(instance) AS elementId, instance.rdfs__label AS courseName
"""
results = neo4j_graph.run(query).data()

# Khởi tạo PhoBERT để xử lý ngôn ngữ tự nhiên
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2")
model = AutoModel.from_pretrained("vinai/phobert-base-v2")

# Hàm mã hóa câu sử dụng PhoBERT
def encode_text(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1)

# Nhận câu truy vấn từ người dùng
user_query = "tìm kiếm môn học trí tuệ nhân tạo"
encoded_query = encode_text(user_query)

# Tính toán độ tương đồng cosine
def cosine_similarity(tensor1, tensor2):
    return torch.nn.functional.cosine_similarity(tensor1, tensor2).item()

# Tìm kiếm và tính toán độ tương đồng
for result in results:
    course_name = result['courseName']
    encoded_course = encode_text(course_name)
    similarity = cosine_similarity(encoded_query, encoded_course)
    print(f"Mã môn học: {result['code']}, Tên môn học: {course_name}, Element ID: {result['elementId']}, Độ tương đồng: {similarity:.4f}")


Some weights of RobertaModel were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Mã môn học: mm48, Tên môn học: Lập trình hệ thống nhúng, Element ID: 3, Độ tương đồng: 0.5794
Mã môn học: mm31, Tên môn học: Giao tiếp người máy, Element ID: 11, Độ tương đồng: 0.5469
Mã môn học: mm49, Tên môn học: Lập trình di động, Element ID: 26, Độ tương đồng: 0.5724
Mã môn học: mm51, Tên môn học: Lập trình Python, Element ID: 33, Độ tương đồng: 0.5214
Mã môn học: mm27, Tên môn học: Lập trình Java, Element ID: 50, Độ tương đồng: 0.5310
Mã môn học: mm18, Tên môn học: Cơ sở dữ liệu, Element ID: 51, Độ tương đồng: 0.5834
Mã môn học: mm23, Tên môn học: Mạng máy tính, Element ID: 53, Độ tương đồng: 0.5661
Mã môn học: mm24, Tên môn học: Phân tích và thiết kế hướng đối tượng, Element ID: 58, Độ tương đồng: 0.6478
Mã môn học: mm40, Tên môn học: Lập trình trên Linux, Element ID: 59, Độ tương đồng: 0.5772
Mã môn học: mm37, Tên môn học: Lập trình mạng, Element ID: 61, Độ tương đồng: 0.5800
Mã môn học: mm45, Tên môn học: Quản lý dự án Công nghệ thông tin, Element ID: 62, Độ tương đồng: 0.6488
